### Let's install some requirements and set some variables:

In [ ]:
#!pip install mysql-connector boto3

In [ ]:
import mysql.connector

pubEndpoint = "35.172.134.237"
lambdaRole = "arn:aws:iam::609170348072:role/cfst-1048-7c98d29a9eddbf3bbde89db39425-ALambdaRole-TWCPES760GI4"
endpoint = "10.0.0.121"
username = "cloud_user"
password = "linuxacademy"
dbname = "moviesdb"
tablename = "Movies"

### Now, we will establish a connection to the server and create our database:

In [ ]:

mydb = mysql.connector.connect(
  host= endpoint,
  user= username,
  passwd= password
)

mycursor = mydb.cursor()



In [ ]:
mycursor = mydb.cursor()
mycursor.execute("CREATE DATABASE " + dbname)

### Let's view the databases to ensure it was created:

In [ ]:
mycursor.execute("SHOW DATABASES")

for x in mycursor:
  print(x)

### Now to connect to our database and create our table:

In [ ]:
mydb = mysql.connector.connect(
  host= endpoint,
  user= username,
  passwd= password,
  database= dbname
)

mycursor = mydb.cursor()

In [ ]:
mycursor.execute("CREATE TABLE " + tablename + " (Year INT, Title VARCHAR(255), Actor VARCHAR(255), Rating INT, Runtime INT, Uploaded VARCHAR(255))")

In [ ]:
mycursor.execute("ALTER TABLE " + tablename + " ADD COLUMN id INT AUTO_INCREMENT PRIMARY KEY")

### Let's add a sample record to ensure the database is setup properly:

In [ ]:
sql = "INSERT INTO " + tablename + " (Year, Title, Actor, Rating, Runtime, Uploaded) VALUES (%s, %s, %s, %s, %s, %s)"
val = ("1989", "Die Hard", "Bruce Willis", "10", "7920", "Yes")
mycursor.execute(sql, val)

mydb.commit()

print(mycursor.rowcount, "record inserted.")

In [ ]:
mydb.commit()
mycursor = mydb.cursor()
mycursor.execute("SELECT * FROM " + tablename)

myresult = mycursor.fetchall()

for x in myresult:
  print(x)

#### Now, head back to the console and create the Lambda function, the table, and the stream. Then we'll finish up!

In [ ]:
import boto3
client = boto3.client('lambda',  region_name='us-east-1')


response = client.create_function(
    FunctionName='ddbStream',
    Runtime='nodejs4.3',
    Role= lambdaRole,
    Handler='ddbStream.handler',
    Code={
        'ZipFile': open('ddbStream.zip', 'rb').read()
    },
    Description='Extracts from DynamoDB Stream and adds to Relational DB',
    Timeout=5,
    Environment={
        'Variables': {
            'endPoint': pubEndpoint
        }
    }
)

In [ ]:
# !aws lambda create-event-source-mapping \
#     --region us-east-1 \
#     --function-name ddbStream \
#     --event-source arn:aws:dynamodb:us-east-1:609170348072:table/Movies/stream/2018-09-26T21:12:38.827 \
#     --batch-size 1 \
#     --starting-position TRIM_HORIZON

### Finally, let's add 100 records to our DynamoDB table and ensure the streaming works!

In [ ]:
from __future__ import print_function # Python 2/3 compatibility
import boto3
import json
import decimal
import sys
import random
dynamodb = boto3.resource('dynamodb',  region_name='us-east-1')
table = dynamodb.Table('Movies')

choices = ['yes', 'no']
i = 0
# with open("/home/cloud_user/moviedata.json") as json_file:
with open("moviedata.json") as json_file:
    with table.batch_writer() as batch:
        movies = json.load(json_file, parse_float = decimal.Decimal)
        for movie in movies:
            i = i + 1
            if i == 101:
                break
            year = int(movie['year'])
            title = movie['title']
            star = movie['actors'][0]
            rating = movie['rating']
            running_time = movie['running_time_secs']
            uploaded = random.choice(choices)

            print("Adding movie:", year, title, star, rating, running_time, uploaded)

            batch.put_item(
               Item={
                   'year': year,
                   'title': title,
                   'actor': star,
                   'rating': rating,
                   'running_time': running_time,
                   'uploaded' : uploaded
                }
            )


In [ ]:
import pandas as pd

mydb.commit()
mycursor = mydb.cursor()
mycursor.execute("SELECT * FROM " + tablename)

myresult = mycursor.fetchall()

item_list = []
for i in myresult:
    item = {'id':i[5], 
            'title':i[0], 
            'actor' :i[1], 
            'rating' :i[2], 
            'running_time' :i[3], 
            'uploaded' :i[4],
            'year' :i[6]}
    item_list.append(item)
df = pd.DataFrame(data=item_list,columns=['id','year','title','actor','running_time','rating','uploaded'])
df.head(30)

### Troubleshooting
The snippets below can help reset things to default or troubleshoot.

##### Delete all items in the relational database:

In [ ]:
mycursor = mydb.cursor()
mycursor.execute("TRUNCATE TABLE " + tablename)

### All Done! Awesome Job!